In [ ]:
import torch as tn
from torchvision import datasets, transforms
import torchtt as tntt
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [ ]:
device_name = 'cuda:0'
data_dir_test = 'seg_test/'
data_dir_train = 'seg_train/'
N_shape = [15,10]

In [ ]:
transform_train = transforms.Compose([transforms.Resize(N_shape[0]*N_shape[1]), transforms.CenterCrop(N_shape[0]*N_shape[1]), transforms.ToTensor()]) #, transforms.Normalize(tn.tensor([0.4885, 0.4525, 0.4163]), tn.tensor([0.2549, 0.2476, 0.2495]))])
dataset_train = datasets.ImageFolder(data_dir_train, transform=transform_train)
dataloader_train = tn.utils.data.DataLoader(dataset_train, batch_size=100, shuffle=True)

transform_test = transforms.Compose([transforms.Resize(N_shape[0]*N_shape[1]), transforms.CenterCrop(N_shape[0]*N_shape[1]), transforms.ToTensor()]) #, transforms.Normalize(tn.tensor([0.4885, 0.4525, 0.4163]), tn.tensor([0.2549, 0.2476, 0.2495])) ])
dataset_test = datasets.ImageFolder(data_dir_test, transform=transform_test)
dataloader_test = tn.utils.data.DataLoader(dataset_test, batch_size=100, shuffle=True)

# inputs_train = list(dataloader_train)[0][0].to(device_name)
# labels_train = list(dataloader_train)[0][1].to(device_name)
# 
# inputs_test = list(dataloader_test)[0][0].to(device_name)
# labels_test = list(dataloader_test)[0][1].to(device_name)

In [ ]:
class BasicTT(nn.Module):
    def __init__(self):
        super().__init__()
        self.ttl1 = tntt.nn.LinearLayerTT([3]+N_shape+N_shape, [10]+N_shape+N_shape, [1,6,2,2,2,1])
        self.ttl2 = tntt.nn.LinearLayerTT([10]+N_shape+N_shape, [8,8,8,8,8], [1,2,2,2,2,1])
        self.ttl3 = tntt.nn.LinearLayerTT([8,8,8,8,8], [4,4,4,4,4], [1,2,2,2,2,1])
        self.ttl4 = tntt.nn.LinearLayerTT([4,4,4,4,4], [4,4,4,4,4], [1,2,2,2,2,1])
        self.ttl5 = tntt.nn.LinearLayerTT([4,4,4,4,4], [3,3,3,3,3], [1,2,2,2,2,1])
        self.linear = nn.Linear(3**5, 6, dtype = tn.float32)
        self.logsoftmax = nn.LogSoftmax(1)

    def forward(self, x):
        x = self.ttl1(x)
        x = tn.relu(x)
        x = self.ttl2(x)
        x = tn.relu(x)
        x = self.ttl3(x)
        x = tn.relu(x)
        x = self.ttl4(x)
        x = tn.relu(x)
        x = self.ttl5(x)
        x = tn.relu(x)
        x = x.view(-1,3**5)
        x = self.linear(x)
        return self.logsoftmax(x)



In [ ]:
model = BasicTT()        
model.to(device_name)

# optimizer = tn.optim.SGD(model.parameters(), lr=0.0000001, momentum=0.9)
optimizer = tn.optim.Adam(model.parameters(), lr=0.001)
loss_function = tn.nn.CrossEntropyLoss()

In [ ]:
def do_epoch(i):
    
    loss_total = 0.0
    
    for k, data in enumerate(dataloader_train):
        # tme = datetime.datetime.now()
        inputs, labels = data[0].to(device_name), data[1].to(device_name)
        # tme = datetime.datetime.now() - tme
        # print('t1',tme)
        
        # tme = datetime.datetime.now()
        inputs = tn.reshape(inputs,[-1,3]+2*N_shape)
        # tme = datetime.datetime.now() - tme
        # print('t2',tme)
        
        # tme = datetime.datetime.now()
        optimizer.zero_grad()
        # Make predictions for this batch
        outputs = model(inputs)
        # Compute the loss and its gradients
        loss = loss_function(outputs, labels)
        loss.backward()
        # Adjust learning weights
        optimizer.step()
        # tme = datetime.datetime.now() - tme
        # print('t3',tme)
        
        loss_total += loss.item()
        # print('\t\tbatch %d error %e'%(k+1,loss))
    return loss_total/len(dataloader_train)

def test_loss():
    loss_total = 0 
    for data in dataloader_test:
        inputs, labels = data[0].to(device_name), data[1].to(device_name)
        inputs = tn.reshape(inputs,[-1,3]+2*N_shape)
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss_total += loss.item()
        
    return loss_total/len(dataloader_test)

def test_accuracy():
    n_total = 0 
    n_correct = 0
    
    for data in dataloader_test:
        inputs, labels = data[0].to(device_name), data[1].to(device_name)
        inputs = tn.reshape(inputs,[-1,3]+2*N_shape)
        outputs = model(inputs)
        n_correct += tn.sum(tn.max(outputs,1)[1] == labels)   
        n_total+=inputs.shape[0]
        
    return n_correct/n_total

In [ ]:
n_epochs = 100

history_test = []
history_train = []
for epoch in range(n_epochs):
    print('Epoch ',epoch+1)
    time_epoch = datetime.datetime.now()
    
    model.train(True)
    average_loss = do_epoch(epoch)
    model.train(False)
    
    test_acc = test_accuracy()
    time_epoch = datetime.datetime.now() - time_epoch
    
    print('\tTraining loss %e test accuracy %5.4f'%(average_loss,test_acc))
    print('\tTime for the epoch',time_epoch)
    history_test.append(test_acc)
    history_train.append(average_loss)
    
plt.figure()
plt.plot(np.arange(n_epochs)+1,np.array(history_train))
plt.plot(np.arange(n_epochs)+1,np.array(history_test))
plt.legend(['training','test'])